In [71]:
import numpy as np
import random

In [253]:
class NeuralNet:
    
    #Set up neural net with layers of widths defined by sizes
    #Weights are initialized to zeros
    def __init__(self,sizes):
        self.depth = len(sizes)
        self.sizes = sizes
        self.W = []
        self.B = []
        for i in range(self.depth-1):
            w = np.zeros((sizes[i+1],sizes[i]))
            self.W.append(w)
            b = np.zeros((sizes[i+1],1))
            self.B.append(b)
    
    #Sets the weights to random initial values.
    def setRandom(self):
        sizes=self.sizes
        self.W = []
        self.B = []
        for i in range(self.depth-1):
            w = np.random.randn(sizes[i+1],sizes[i])
            self.W.append(w)
            b = np.random.randn(sizes[i+1],1)
            self.B.append(b)
    
    #Define custom starting weights
    def setManual(self,w,b):
        for i in range(self.depth-1):
            self.W[i] = w[i]
            self.B[i] = b[i]
    
    #Feed input into neural net and return all z and activation layers.
    def feedForward(self,x):
        A = [x]
        Z = []
        for i in range(self.depth-1):
            w = self.W[i]
            #print("w",w)
            b = self.B[i]
            #print("b",b)
            #print("a",A[-1])
            z = np.dot(w,A[-1])+b
            #print("z",z)
            Z.append(z)
            a = sigmoid(z)
            A.append(a)
        return Z,A
    
    #Do backward propagation to get gradients of all weights
    def backProp(self,x,y):
        gradB = [np.zeros(b.shape) for b in self.B]
        gradW = [np.zeros(w.shape) for w in self.W]
        Z,A = self.feedForward(x)
        
        delta = (Z[-1]-y)
        #print("d",delta)
        gradB[-1] = delta
        gradW[-1] = np.dot(delta,A[-2].T)
        for l in range(2,self.depth):
            z = Z[-l]
            #print("z",z)
            sp = dSigmoid(z)
            #print("sp",sp)
            delta = np.dot(self.W[-l+1].T,delta)*sp
            #print("d",delta)
            gradB[-l] = delta
            #print("a",A[-l-1])
            gradW[-l] = np.dot(delta,A[-l-1].T)
        return gradB,gradW
    
    #Get gradients ot weights, then update weights
    def update(self,x,y,g):
        gradB = [np.zeros(b.shape) for b in self.B]
        gradW = [np.zeros(w.shape) for w in self.W]
        dgradB,dgradW = self.backProp(x,y)
        gradB = [gb+dgb for gb,dgb in zip(gradB,dgradB)]
        gradW = [gw+dgw for gw,dgw in zip(gradW,dgradW)]
        self.W = [w-g*gw for w,gw in zip(self.W,gradW)]
        self.B = [b-g*gb for b,gb in zip(self.B,gradB)]
    
    #Check how well the Neural Net works
    def evaluate(self,testData):
        count = 0
        for x,y in testData:
            Z,A = self.feedForward(np.atleast_2d(x).T)
            y0 = Z[-1][0,0]
            #print(y0,y)
            if(round(y0) == round(y)):
                count += 1
        return count
    
    #Stochastic Graident Descent method to learn weights
    def SGD(self,trainData,epochs):
        n = len(trainDataY)
        for i in range(epochs):
            random.sample(trainData,n)
            for x,y in trainData:
                self.update(np.atleast_2d(x).T,y,gamma(0.5,i,10))
            print("Epoch {0} complete".format(i+1))



In [254]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def dSigmoid(z):
    return sigmoid(z)*(1-sigmoid(z))

def gamma(g0,t,d):
    return g0/(1.0+t*g0/d)

In [260]:
trainDataRaw = np.genfromtxt("bank-note/train.csv",delimiter=",")
trainDataX = trainDataRaw[:,:-1]
trainDataY = trainDataRaw[:,-1]
trainData = list(zip(trainDataX,trainDataY))
w = len(trainDataX[0])

#Set Up Neural Network with layers of length w,W,W,W, and 1.
#Max Width (W) of 25
W = 10
NN = NeuralNet([w,W,W,W,1])
#Comment Out Below Line to use zeros as starting weights. Otherwise, random.
NN.setRandom()

#E = epochs to train over
E = 3
NN.SGD(trainData,E)
print(NN.evaluate(trainData),"/",len(trainData))

testDataRaw = np.genfromtxt("bank-note/test.csv",delimiter=",")
testDataX = testDataRaw[:,:-1]
testDataY = testDataRaw[:,-1]
testData = list(zip(testDataX,testDataY))

print(NN.evaluate(testData),"/",len(testData))

Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
872 / 872
500 / 500
